In [101]:
import os
os.chdir('/data/hyeryung/mucoco')

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from new_module.em_training.nli.train_modules import load_config
from new_module.em_training.nli.data_handling import NLI_Dataset, NLI_BatchSampler, NLI_DataLoader, load_nli_data, load_additional_nli_training_data
import pandas as pd

import torch

In [118]:
import new_module.em_training.nli.data_handling
import importlib
importlib.reload(new_module.em_training.nli.data_handling)
from new_module.em_training.nli.data_handling import NLI_DataLoader, NLI_Dataset

In [3]:
config = load_config('new_module/em_training/config.yaml')
config['device'] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

In [2]:
batch_size = 32

train_dev_data = load_nli_data()
add_train_data = load_additional_nli_training_data()
add_train_data = add_train_data.groupby('binary_labels').sample(5000)

train_dev_data = pd.concat([train_dev_data, add_train_data], axis=0)

Preprocessed dataset already exists. Loading data from file
Preprocessed dataset already exists. Loading data from file


In [5]:
train_data = train_dev_data.loc[train_dev_data['split'] == 'train'].copy()
dev_data = train_dev_data.loc[train_dev_data['split'] == 'dev'].copy()

In [6]:
train_dev_data['finegrained_labels'].isna().value_counts()

finegrained_labels
False    56092
True     10000
Name: count, dtype: int64

In [7]:
config['energynet']['batch_size']

16

In [107]:
train_dataset = NLI_Dataset(train_data, 'binary_labels')
dev_dataset = NLI_Dataset(dev_data, 'binary_labels')

In [119]:
train_dataloader = NLI_DataLoader(train_dataset, config, 'train', tokenizer, allow_oversample=True).get_dataloader()

In [104]:
for key, value in train_dataloader.batch_sampler.data.items():
    print(key, "-->", len(value))

nan --> 10000
0 --> 29889
(0, 0.5) --> 5058
[0.5, 1) --> 4912
1 --> 10622


In [105]:
print(train_dataloader.batch_sampler.num_finegrained_batches)
print(train_dataloader.batch_sampler.num_nan_batches)

4981
625


In [86]:
import math
for _ in range(3):
    for i, batch in enumerate(train_dataloader):
        # print(i)
        if i == 0:
            print(batch)
        # if i < train_dataloader.batch_sampler.num_nan_batches:
        #     continue
        # else:
        #     # if all([not math.isnan(x) for x in batch['finegrained_labels']]):
            
        #     print(i, batch)
        #     break

[59260, 54253, 54926, 50981, 51664, 50928, 54998, 50512, 55177, 50600, 54486, 59143, 59987, 52501, 57576, 59429]
[58618, 50669, 58506, 57020, 58341, 53788, 51301, 51174, 54621, 55363, 58958, 57374, 51624, 59719, 53253, 60008]
[58994, 57022, 57798, 60058, 50557, 58523, 58677, 55540, 57332, 57270, 55581, 54058, 54879, 50719, 53130, 55840]


In [120]:
import math
for _ in range(3):
    for i, batch in enumerate(train_dataloader):
        # print(i)
        if i == 0:
            print(batch)
            break
        # if i < train_dataloader.batch_sampler.num_nan_batches:
        #     continue
        # else:
        #     # if all([not math.isnan(x) for x in batch['finegrained_labels']]):
            
        #     print(i, batch)
        #     break

{'input_ids': tensor([[   0,    0,  250,  ...,    1,    1,    1],
        [   0,    0, 9058,  ...,    1,    1,    1],
        [   0,    0,  133,  ...,    1,    1,    1],
        ...,
        [   0,    0,  133,  ...,    1,    1,    1],
        [   0,    0,  133,  ...,    1,    1,    1],
        [   0,    0,  250,  ...,    1,    1,    1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([[1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.]], device='cuda:0'), 'finegrained_labels': tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       device='cuda:0')}

In [110]:
dev_dataloader = NLI_DataLoader(dev_dataset, config, 'dev', tokenizer, allow_oversample=True).get_dataloader()

In [111]:
for key, value in dev_dataloader.batch_sampler.data.items():
    print(key, "-->", len(value))

nan --> 0
0 --> 3270
(0, 0.5) --> 1090
[0.5, 1) --> 1090
1 --> 3270


In [112]:
print(dev_dataloader.batch_sampler.num_finegrained_batches)
print(dev_dataloader.batch_sampler.num_nan_batches)

545
0


In [100]:
import math
for _ in range(3):
    for i, batch in enumerate(dev_dataloader):
        # print(i)
        if i == 0:
            print(batch)
        # if len(batch) < config['energynet']['batch_size']:
        #     print(i, batch)
        # if i < train_dataloader.batch_sampler.num_nan_batches:
        #     continue
        # else:
        #     # if all([not math.isnan(x) for x in batch['finegrained_labels']]):
            
        #     print(i, batch)
        #     break

[0, 2, 3, 6, 10, 11, 1, 5, 7, 9, 4, 8, 24, 31, 33, 35]
[0, 2, 3, 6, 10, 11, 1, 5, 7, 9, 4, 8, 24, 31, 33, 35]
[0, 2, 3, 6, 10, 11, 1, 5, 7, 9, 4, 8, 24, 31, 33, 35]


In [113]:
import math
for _ in range(3):
    for i, batch in enumerate(dev_dataloader):
        # print(i)
        if i == 0:
            print(batch)
        # if len(batch) < config['energynet']['batch_size']:
        #     print(i, batch)
        # if i < train_dataloader.batch_sampler.num_nan_batches:
        #     continue
        # else:
        #     # if all([not math.isnan(x) for x in batch['finegrained_labels']]):
            
        #     print(i, batch)
        #     break

{'input_ids': tensor([[    0,     0,  1185,     8,   110,   964,    32,    45,  2814,   259,
             6,    26,  1608, 12170,     4,     2, 14696, 12170,    26,     5,
            82,    58,    45,  2814,    89,     4,     2,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,     0,   970,    32,    10,   346,     9,  3214, 11449,     8,
            97,  4053,    12,  3743,  6464,     6,    70,    19,  3057,  7663,
            11,   382,  1932,    36, 15593,    12,  3743,  3057,   531,   460,
            28,  1199,    13,    11,  1093,  2593,   322,     2, 44948,   523,
          1506,     8,  4053,    12,  3743,  6464,    32,    41,  2679,   317,
             7,   907,  3057,     4,     2,     2,     1,     1],
        [    0,     0,   894,    56,   393,  1299,   357,     4,